# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Liquidation.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 Liquidation

**This is incomplete - a Work In Progress. Do not rely on anything in here**

Mango Markets margin accounts must hold more assets than liabilities.

If a margin account doesn't hold assets worth more than 110% of its liabilities, anyone can 'pay off' some of the liabilities in that account. In return, that payer receives 105% of their tokens back.

This process - called '_liquidation_' - continues until the remaining assets are worth more than 120% of the remaining liabilities, or until there are no more collateral tokens to return to liquidators.


In [ ]:
import time

from solana.publickey import PublicKey
from solana.rpc.commitment import Single
from solana.transaction import Transaction

from Classes import Group, MarginAccount
from Context import Context
from Instructions import ForceCancelOrdersInstructionBuilder, InstructionBuilder, LiquidateInstructionBuilder
from Wallet import Wallet


## Collateralisation Ratios

When trading with leverage on Mango Markets, you start by adding assets to your margin account. Your margin account must hold assets worth more than 120% of the liabilities. This ratio of assets to liabilities is called your '_collateralisation ratio_'.

If a new trade would mean your assets would be worth less 120% of the libilities - that your collateralisation ratio would be less than 120% - you will not be allowed to place the trade. No existing trades are closed, you just can't place any new orders.

The value of assets and liabilities is derived from the prices from on-chain oracles.

Prices change, so even if no new trades are made the value of assets can fall and the liabilities can increase, changing the collateralisation ratio.

**If the collateralisation ratio falls below 110%, your margin account can be '_liquidated_', resulting in the loss of some or a all assets in the account.**


## Safety

Liquidation is how Mango Markets protects accounts from systemic losses.

If the collateralisation ratio were allowed to fall below 100%, that would represent a systemic risk to Mango Markets. Funds would have to be taken from the lending pools, affecting all lenders, which may in turn affect their collateralisation ratio and expose them to liquidation, possibly leading to a cascade of liquidations until the system stabilises.

Liquidators promptly stepping in to 'buy' liquidatable accounts before they fall to less than 100% collateralisation prevent this instability.


# 💧 Liquidation Process

## Steps

The liquidation process involves paying off some or all of one of the liabilities in the under-collateralised margin account. Margin accounts can have liabilities in multiple tokens, but liquidation can only be performed on one token at a time.

These are the basic steps to liquidate an account:
1. Force cancellation of all outstanding orders for the margin account in that market.
2. Build and send the `PartialLiquidate` instruction.
3. Repeat step 2 (if necessary) with fresh tokens.

But to actually run a liquidator, there are probably some additional steps:
1. Find all liquidatable margin accounts.
2. Pick the most appropriate of these margin accounts, based on that account's collatoralisation and the liquidator's token balances.
3. Pick the market with the most value in the margin account's openorders accounts.
4. Force cancellation of all outstanding orders for the margin account in that market.
5. Pick the market with the highest borrows and lowest deposits for the account being liquidated.
6. Build and send the `PartialLiquidate` instruction.
7. Convert the received tokens to your desired tokens.
8. Repeat from step 2 (if necessary) with fresh tokens.

(This is probably not right - it depends on a polling loop. We'll probably have a websocket connection for prices and accounts. But then again it may apply on a price change.)


In [ ]:
class Liquidator:
    def __init__(self, context: Context, wallet: Wallet, group: Group):
        self.context = context
        self.wallet = wallet
        self.group = group

    def liquidate(self, margin_account: MarginAccount) -> str:
        force_cancel_orders_instructions: typing.List[InstructionBuilder] = []
        for index, market_metadata in enumerate(self.group.markets):
            open_orders = margin_account.open_orders_accounts[index]
            if open_orders is not None:
                market = market_metadata.fetch_market(self.context)
                orders = market.load_orders_for_owner(margin_account.owner)
                order_count = len(orders)
                if order_count > 0:
                    force_cancel_orders_instructions += ForceCancelOrdersInstructionBuilder.multiple_instructions_from_margin_account_and_market(default_context, group, wallet, margin_account, market_metadata, order_count)

        transaction = Transaction()
        [transaction.add(builder.build()) for builder in force_cancel_orders_instructions]

        liquidate_instruction = LiquidateInstructionBuilder.from_margin_account_and_market(default_context, group, wallet, margin_account, prices)
        transaction.add(liquidate_instruction.build())

        for instruction in transaction.instructions:
            print("INSTRUCTION")
            print("    Keys:")
            for key in instruction.keys:
                print("        ", f"{key.pubkey}".ljust(45), f"{key.is_signer}".ljust(6), f"{key.is_writable}".ljust(6))
            print("    Data:", " ".join(f"{x:02x}" for x in instruction.data))
            print("    Program ID:", instruction.program_id)

        transaction_response = self.context.client.send_transaction(transaction, wallet.account)
        return transaction_response["result"]



# 🏃 Running

In [ ]:
MARGIN_ACCOUNT_TO_LIQUIDATE = ""

In [ ]:
if __name__ == "__main__":
    if MARGIN_ACCOUNT_TO_LIQUIDATE == "":
        raise Exception("No margin account to liquidate - try setting the variable MARGIN_ACCOUNT_TO_LIQUIDATE to a margin account public key.")

    import os.path
    from Context import default_context

    filename = "id.json"

    if not os.path.isfile(filename):
        print(f"Wallet file '{filename}' is not present.")
    else:
        wallet = Wallet.load(filename)
        group = Group.load(default_context)
        print("Before Balances:")
        print(f"    SOL balance: {default_context.fetch_sol_balance(wallet.address):>18,.8f}")
        group = Group.load(default_context)
        for token in group.tokens:
            print(f"{token.name:>7} balance: {default_context.fetch_token_balance(wallet.address, token.mint):>18,.8f}")

        prices = group.get_prices()
        margin_account = MarginAccount.load(default_context, group, PublicKey(MARGIN_ACCOUNT_TO_LIQUIDATE))
        print("Before", margin_account.get_balance_sheet_totals(group, prices))
        liquidator = Liquidator(default_context, wallet, group)
        transaction_id = liquidator.liquidate(margin_account)
        print("Transaction ID:", transaction_id)
        print("Waiting for confirmation...")

        for wait in range(0, 60):
            time.sleep(1)
            confirmed = default_context.client.get_confirmed_transaction(transaction_id)
            if confirmed["result"] is not None:
                print(f"Confirmed after {wait} seconds.")
                break

        margin_account_adfter_liquidation = MarginAccount.load(default_context, group, PublicKey(MARGIN_ACCOUNT_TO_LIQUIDATE))
        print("After", margin_account_adfter_liquidation.get_balance_sheet_totals(group, prices))
        print("After Balances:")
        print(f"    SOL balance: {default_context.fetch_sol_balance(wallet.address):>18,.8f}")
        group = Group.load(default_context)
        for token in group.tokens:
            print(f"{token.name:>7} balance: {default_context.fetch_token_balance(wallet.address, token.mint):>18,.8f}")
